In [10]:
# import libraries
import random
import time
import threading
import os
import mplfinance as mpf
import pandas as pd
import numpy as np
import datetime as dt
import botlogic as bl
import ipywidgets as widgets
from prettytable import PrettyTable
from ipywidgets import AppLayout
from IPython.display import display


# Creating a "comms" folder which is used as a dependency of this code in order to create/update visualisations
folder_path = 'comms'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Market infrastructure
def buy_orderbook_append (timestamp, trader_id, quantity, price): 
    global buy_orderbook
    new_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : quantity, "Price" : price})
    buy_orderbook = pd.concat([buy_orderbook, new_order.to_frame().T], ignore_index=True)
    buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)

def sell_orderbook_append (timestamp, trader_id, quantity, price):
    global sell_orderbook 
    new_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : quantity, "Price" : price})
    sell_orderbook = pd.concat([sell_orderbook, new_order.to_frame().T], ignore_index=True)
    sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)

def transaction_append (timestamp, buy_id, sell_id, quantity, price, aggressor_id):
    global transaction_log
    transaction = pd.Series({"Timestamp" : timestamp, "Buy_Side_Trader_ID" : buy_id, "Sell_Side_Trader_ID" : sell_id, "Quantity" : quantity, "Price" : price, "Aggressor_ID" : aggressor_id})
    transaction_log = pd.concat([transaction_log, transaction.to_frame().T], ignore_index=True)
    transaction_log["Timestamp"] = pd.to_datetime(transaction_log["Timestamp"])
    transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)

# Pulling key figures
class market_key_figures:
    def __init__(self):
        global transaction_log, buy_orderbook, sell_orderbook
        # tests to calculate figs from transaction log
        try:
            transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)
            self.market_price = transaction_log["Price"].iloc[-1]
            self.open_price = transaction_log["Price"].iloc[0]
            self.abs_price_mvmt = round(self.market_price - self.open_price,2)
            self.per_price_mvmt = round((self.abs_price_mvmt/self.open_price)*100,2)

            transaction_log.sort_values(by=["Price"], ascending=[True], inplace = True)
            self.price_max = transaction_log["Price"].iloc[-1]
            self.price_min = transaction_log["Price"].iloc[0]
            self.price_range = round(self.price_max - self.price_min,2)

            transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)
            self.transact_test = "pass"
        except:
            self.transact_test = "fail"

        # tests to calculate figs from orderbook
        try: 
            buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
            self.best_bid = buy_orderbook['Price'].iloc[0]
            self.buy_orderbook_test = "b_pass"
        except:
            self.buy_orderbook_test = "b_fail"

        try:
            sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
            self.best_ask = sell_orderbook['Price'].iloc[0]
            self.sell_orderbook_test = "s_pass"
        except:
            self.sell_orderbook_test = "s_fail"
      
        # combining these tests to produce a single outcome result 
        if self.transact_test == "pass" and (self.buy_orderbook_test == "b_pass" and self.sell_orderbook_test == "s_pass"):
            self.key_figs_test = "pass"
        elif self.transact_test == "fail" and (self.buy_orderbook_test == "b_pass" and self.sell_orderbook_test == "s_pass"):
            self.key_figs_test = "o_semipass"
        elif self.transact_test == "pass" and (self.buy_orderbook_test == "b_fail" or self.sell_orderbook_test == "s_fail"):
            self.key_figs_test = "t_semipass"

        #elif self.transact_test == "fail" and (self.buy_orderbook_test == "b_fail" or self.sell_orderbook_test == "s_fail"):
        else: 
            self.key_figs_test = "fail"

# Order Matching & transaction log
def orderbook_matching (input_order):
    global df_participants, transaction_log, buy_orderbook, sell_orderbook
    key_figs =market_key_figures()
    order_price = round(input_order["Price"],2)

    if order_price > 0:
        bid_ask_flag = input_order["Flag"]
        order_quantity = int(input_order["Quantity"])
        order_id = input_order["Trader_ID"]
        timestamp = dt.datetime.now()

        def bot_crediting (transaction_price, transaction_qty, buy_ID, sell_ID):
            global df_participants
            df_participants.loc[df_participants["Trader_ID"] == buy_ID, "Asset"] += transaction_qty
            transaction_val = transaction_price * transaction_qty
            df_participants.loc[df_participants["Trader_ID"] == sell_ID, "Wealth"] += transaction_val    

        # Not the cleanest code, but it works for now. 
        def bid_price_check (order_price, best_ask):
            if order_price >= best_ask:
                check = "True"
            else:
                check = "False"
            return check
        def ask_price_check (order_price, best_bid):
            if order_price <= best_bid:
                check = "True"
            else:
                check = "False"
            return check
        
        try:
            bprice_check = bid_price_check(order_price, key_figs.best_ask)
        except:
            bprice_check = "False"
        
        try:
            aprice_check = ask_price_check(order_price, key_figs.best_bid)
        except:
            aprice_check = "False"

        if bid_ask_flag == "bid" and order_price >= key_figs.best_ask:
            # Pulling sell orderbook info
            best_sell_id = sell_orderbook['Trader_ID'].iloc[0]
            best_sell_qty = sell_orderbook['Quantity'].iloc[0]

            # Running a while loop to execute the order against as many eligible orders 
            while order_quantity > 0 and (key_figs.sell_orderbook_test =="s_pass" and bprice_check == "True"):
                result = order_quantity - best_sell_qty

                if result >= 0:
                    # this will clear the sell order as it can be executed against in its full entirety 
                    transaction_append(timestamp, order_id, best_sell_id, best_sell_qty, order_price, order_id)

                    # removing cleared sell order from orderbook, and resets index
                    sell_orderbook = sell_orderbook.iloc[1:]
                    sell_orderbook.reset_index(drop = True, inplace=True)
                    order_quantity = result
                    bot_crediting(order_price, best_sell_qty, order_id, best_sell_id)

                elif result < 0:
                    # this clears the bid order against the sell order with left over quantity, writing the new quantity to the sell orderbook 
                    transaction_append(timestamp, order_id, best_sell_id, order_quantity, order_price, order_id)
                    bot_crediting(order_price, order_quantity, order_id, best_sell_id)

                    # updating the quantity of the partially-filled sell order
                    new_sell_qty = best_sell_qty - order_quantity
                    sell_orderbook.at[0, "Quantity"] = new_sell_qty
                    order_quantity = 0

                # updating best ask info 
                try:    
                    key_figs.best_ask = sell_orderbook['Price'].iloc[0]
                    best_sell_id = sell_orderbook['Trader_ID'].iloc[0]
                    best_sell_qty = sell_orderbook['Quantity'].iloc[0]
                    bprice_check = bid_price_check(order_price, key_figs.best_ask)

                except:        
                    if order_quantity > 0:
                        buy_orderbook_append(timestamp, order_id, order_quantity, order_price)
                    break

        elif bid_ask_flag == "ask" and order_price <= key_figs.best_bid:   
            # Pulling buy orderbook info
            best_buy_id = buy_orderbook['Trader_ID'].iloc[0]
            best_buy_qty = buy_orderbook['Quantity'].iloc[0]

            # Running a while loop to execute the order against as many eligible orders 
            while order_quantity > 0 and (key_figs.buy_orderbook_test == "b_pass" and aprice_check == "True"):

                result = order_quantity - best_buy_qty

                if result >= 0:
                    # this will clear the buy order as it can be executed against in its full entirety 
                    transaction_append(timestamp, best_buy_id, order_id, best_buy_qty, order_price, order_id)
      
                    # removing cleared buy order from orderbook, and resets index
                    buy_orderbook = buy_orderbook.iloc[1:]
                    buy_orderbook.reset_index(drop = True, inplace=True)
                    order_quantity = result
                    bot_crediting(order_price, best_buy_qty, best_buy_id, order_id)  

                elif result < 0:
                    # this clears the ask order against the buy order with left over quantity, writing the new quantity to the buy orderbook 
                    transaction_append(timestamp, best_buy_id, order_id, order_quantity, order_price, order_id)
                    bot_crediting(order_price, order_quantity, best_buy_id, order_id)

                    # updating the quantity of the partially-filled buy order
                    new_buy_qty = best_buy_qty - order_quantity
                    buy_orderbook.at[0, "Quantity"] = new_buy_qty
                    order_quantity = 0

                # updating best bid info
                try:
                    key_figs.best_bid = buy_orderbook['Price'].iloc[0]
                    best_buy_id = buy_orderbook['Trader_ID'].iloc[0]
                    best_buy_qty = buy_orderbook['Quantity'].iloc[0]
                    aprice_check = ask_price_check(order_price, key_figs.best_bid)

                except:
                    if order_quantity > 0:
                        sell_orderbook_append(timestamp, order_id, order_quantity, order_price)   
                    break


        # running the price checks again, to see if the order can match against anything else
        try:
            bprice_check = bid_price_check(order_price, key_figs.best_ask)
        except:
            bprice_check = "False"
        
        try:
            aprice_check = ask_price_check(order_price, key_figs.best_bid)
        except:
            aprice_check = "False"
        # any orders that do not match the best bid are appended to the sell orderbook
        if (bid_ask_flag == "ask" and aprice_check == "False") and order_quantity > 0:
            sell_orderbook_append(timestamp, order_id, order_quantity, order_price)
        elif (bid_ask_flag == "bid" and bprice_check == "False") and order_quantity > 0:
            buy_orderbook_append(timestamp, order_id, order_quantity, order_price)

def orderbook_vis (orderbook, side, html_flag):
    agg_orderbook = pd.DataFrame(columns=["Price", "Quantity"])
    agg_orderbook = orderbook.groupby("Price").agg({"Quantity" : 'sum'}).reset_index()
    agg_orderbook.columns = ['Price', 'Total_Quantity']
    
    if html_flag == True:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total_Quantity']
        if side == "b":
            agg_orderbook.sort_values(by=['Price'], ascending=False, inplace=True) 
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total_Quantity']])
        vis = vis.get_html_string()
    elif html_flag == False:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total_Quantity']
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total_Quantity']])
    return vis

# S.1 & S.2 & S.3
def participant_creation (participant_num):
    global df_participants

    participant_num += 1
    i = 1
    profiles = ["IB Trader", "WM Trader", "Market Maker", "Retail Investor", "Private Investor"]
    profile_proportions = [0.50, 0.05, 0.02, 0.33, 0.10]

    # Check to see that the proportions add up to 1
    if not np.isclose(sum(profile_proportions), 1.0):
        raise ValueError("Profile proportions do not sum up to 1.")
    
    # Calculating number of participants for each string, and adjusting to match pariticpant_num
    num_assignments = [int(round(p * participant_num)) for p in profile_proportions]

    total_assigned = sum(num_assignments)
    while total_assigned < participant_num:
        for i in range(len(num_assignments)):
            if total_assigned < participant_num:
                num_assignments [i] += 1
                total_assigned += 1
            else:
                break
    while total_assigned > participant_num:
        for i in range(len(num_assignments)):
            if total_assigned > participant_num and num_assignments[i] > 0:
                num_assignments[i] -= 1
                total_assigned -= 1
            else:
                break
    
    assigned_profiles = []
    for profile, count in zip(profiles, num_assignments):
        assigned_profiles.extend([profile] * count)

    # Validation check against number of participants 
    assigned_profiles = assigned_profiles[:participant_num]
    # Creating unique IDs
    pariticipant_ids = [i for i in range(participant_num)]

    df_participants = pd.DataFrame({
        "Trader_ID" : pariticipant_ids,
        "Asset" : 0,
        "Wealth" : 0,
        "Risk" : 0, 
        "Activity" : 0,
        "Delay" : np.random.poisson(lam=2),
        "Profile" : assigned_profiles
    })

    def generate_ib():
        asset = np.random.randint(500,1000)
        wealth = np.random.randint(5000,10000),
        risk = np.random.uniform(0.01, 0.35)
        activity = np.random.uniform(0.60, 0.90)
        return asset, wealth, risk, activity 
    def generate_wm():
        asset = np.random.randint(400,800)
        wealth = np.random.randint(5000,9000),
        risk = np.random.uniform(0.01, 0.25)
        activity = np.random.uniform(0.01, 0.30)
        return asset, wealth, risk, activity 
    def generate_mm():
        asset = np.random.randint(100,500)
        wealth = np.random.randint(6000,12000),
        risk = np.random.uniform(0.01, 0.10)
        activity = np.random.uniform(0.70, 0.90)
        return asset, wealth, risk, activity 
    def generate_ri():
        asset = np.random.randint(1,200)
        wealth = np.random.randint(500,4000),
        risk = np.random.uniform(0.50, 1.0)
        activity = np.random.uniform(0.30, 0.80)
        return asset, wealth, risk, activity 
    def generate_pi():
        asset = np.random.randint(50,400)
        wealth = np.random.randint(1000,5000),
        risk = np.random.uniform(0.30, 0.70)
        activity = np.random.uniform(0.20, 0.50)
        return asset, wealth, risk, activity 
    
    # Defining the conditions for each profile type
    updates = [
        {'condition': df_participants['Profile'] == "IB Trader", 'generate_values': lambda: generate_ib()},
        {'condition': df_participants['Profile'] == "WM Trader", 'generate_values': lambda: generate_wm()},
        {'condition': df_participants['Profile'] == "Market Maker",'generate_values': lambda: generate_mm()},
        {'condition': df_participants['Profile'] == "Retail Investor",'generate_values': lambda: generate_ri()},
        {'condition': df_participants['Profile'] == "Private Investor",'generate_values': lambda: generate_pi()},
    ]
    # adding the traits for each profile type
    for update in updates:
        for index, row in df_participants.loc[update['condition']].iterrows():
            asset, wealth, risk, activity = update['generate_values']()
            df_participants.at[index, "Asset"] = asset
            df_participants.at[index, "Wealth"] = wealth
            df_participants.at[index, "Risk"] = risk
            df_participants.at[index, "Activity"] = activity

    return df_participants

def user_creation (df_participants, participant_int):
    user_profile = pd.Series({
        "Trader_ID" : participant_int + 1, 
        "Asset" : 100000, 
        "Wealth" : 100000, 
        "Risk" : 0, 
        "Activity" : 0, 
        "Delay" :0, 
        "Profile" : "User"
        })

    df_participants = pd.concat([df_participants, user_profile.to_frame().T], ignore_index=True)
    return df_participants

# S.4 & B.1
def iteration_start():
    df_available = pd.DataFrame()

    bot_profiles = ["IB Trader", "WM Trader", "Market Maker", "Retail Investor", "Private Investor"]
    # Could be slow - look to optimise using vectorization/ Numpy function?
    for index, row in df_participants.iterrows():
        if  row["Profile"] in bot_profiles and row["Delay"] == 0:
            df_available = pd.concat([df_available, row.to_frame().T], ignore_index=True)
            # df_available = df_available.astype({"Trader_ID":int, "Asset":float, "Wealth":float, "Risk":float, "Activity":float, "Delay":int})
            df_available = df_available.sample(frac = 1)
            
    return df_available

# T.1.1: cancelling orders away from priority
def cancel_faraway_orders (bot):
    global transaction_log
    key_figs = market_key_figures()
    if key_figs.key_figs_test == 'pass':
        if len(transaction_log) > 24:
            prev_price = transaction_log.iat[-25,4]
        else:
            prev_price = transaction_log.iat[-1,4]
        market_delta = prev_price - key_figs.market_price

        # checking for any orders that should be cancelled. If they are to be cancelled, the appropriate wealth/asset is credited back to the bot's account.    
        for index, order in buy_orderbook.iterrows():
            market_order_spread = key_figs.best_bid - order["Price"]
            if bot[0] == order["Trader_ID"] and market_order_spread > 0.1: # 10 ticks away from priority
                bot["Wealth"] += order["Price"] * order["Quantity"]
                buy_orderbook.drop(index=index, inplace=True)
            elif bot[0] == order["Trader_ID"] and (market_order_spread <= 0.01 and market_delta > 0.02):           
                bot["Wealth"] += order["Price"] * order["Quantity"]
                buy_orderbook.drop(index=index, inplace=True)
                
        for index, order in sell_orderbook.iterrows():
            order_market_spread = order["Price"] - key_figs.best_ask
            if bot[0] == order[1] and order_market_spread > 0.1:
                bot["Asset"] += order["Quantity"]
                sell_orderbook.drop(index=index, inplace=True)
            elif bot[0] == order[1] and (market_order_spread <= 0.01 and market_delta < -0.02):           
                bot["Asset"] += order["Quantity"]
                sell_orderbook.drop(index=index, inplace=True)
    return bot

# debiting function - takes order info, and debits the money or asset accordinlgy, from the bot
def bot_debiting (bot, price, quantity, result):
    global df_participants
    if result == "buy_order" or result == "buy_execute":
        order_value = price * quantity
        df_participants.loc[df_participants["Trader_ID"] == bot["Trader_ID"], "Wealth"] -= order_value
        
    elif result == "sell_order" or result == "sell_execute":        
        df_participants.loc[df_participants["Trader_ID"] == bot["Trader_ID"], "Asset"] -= quantity
    return bot

def quantity_cap_check(qty):
    if qty > 1000:
        r_qty = 1000
    else:
        r_qty = qty
    return r_qty

# function that decides order quantity and price
def bot_market_interact (result, bot):
    global buy_orderbook, sell_orderbook
    # D.2, D.3, D.4
    key_figs = market_key_figures()
    timestamp = dt.datetime.now()
    trader_id = bot["Trader_ID"]
    max_buy_quantity = quantity_cap_check(bot["Wealth"] / key_figs.market_price)
    max_sell_quantity = quantity_cap_check(bot["Asset"])

    if key_figs.key_figs_test == "pass":
        best_bid_spread = key_figs.market_price - key_figs.best_bid
        best_ask_spread = key_figs.best_ask - key_figs.market_price

        if result == 'buy_order' and best_bid_spread > 0.02:
            #D.3.1
            order_price = round(key_figs.best_bid + 0.01,2)
            order_quantity = round(random.uniform(0.10, 0.25) * max_buy_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

        elif result == 'buy_order' and best_bid_spread <= 0.02:
            #D.3.2
            offset = round(abs(np.random.standard_normal()) * 4) * 0.01 
            order_price = round(key_figs.best_bid - offset,2)
            order_quantity = round(random.uniform(0.10, 0.25) * max_buy_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

        elif result == 'buy_execute':
            #D.5.1
            order_price = key_figs.best_ask
            order_quantity = round(random.uniform(0.10, 0.20) * max_buy_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

        elif result == 'sell_order' and best_ask_spread > 0.02:
            #D.4.1
            order_price = round(key_figs.best_ask - 0.01,2)
            order_quantity = round(random.uniform(0.10, 0.25) * max_sell_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

        elif result == 'sell_order' and best_ask_spread <= 0.02:
            #D.4.2
            offset = round(abs(np.random.standard_normal()) * 4) * 0.01 
            order_price = round(key_figs.best_ask + offset,2)
            order_quantity = round(random.uniform(0.10, 0.25) * max_sell_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

        elif result == 'sell_execute':
            #D.5.2
            order_price = key_figs.best_bid
            order_quantity = round(random.uniform(0.10, 0.20) * max_sell_quantity)
            input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})
            bot = bot_debiting(bot, order_price, order_quantity, result)
            orderbook_matching(input_order)

    # help create liquidity in orderbooks 
    elif (key_figs.key_figs_test == 't_semipass' and key_figs.buy_orderbook_test == "b_fail") and (result == 'buy_order' or result == 'buy_execute'):
        #D.3.1
        offset = random.randint(2 , 10) * 0.01 
        order_price = round(key_figs.best_ask - offset,2)
        order_quantity = round(random.uniform(0.40, 0.75) * max_buy_quantity)

        if order_price > 0:
            bot = bot_debiting(bot, order_price, order_quantity, result)
            buy_orderbook_append(timestamp, trader_id, order_quantity, order_price)
        else:
            order_price = 0.01
            bot = bot_debiting(bot, order_price, order_quantity, result)
            buy_orderbook_append(timestamp, trader_id, order_quantity, order_price)

    elif (key_figs.key_figs_test == 't_semipass' and key_figs.sell_orderbook_test == "s_fail") and (result == 'sell_order' or result == 'sell_execute'):
        offset = random.randint(2 , 10) * 0.01 
        order_price = round(key_figs.best_bid + offset,2)
        order_quantity = round(random.uniform(0.40, 0.75) * max_sell_quantity)

        if order_price > 0:
            bot = bot_debiting(bot, order_price, order_quantity, result)
            sell_orderbook_append(timestamp, trader_id, order_quantity, order_price)
        else:
            order_price = 0.03
            bot = bot_debiting(bot, order_price, order_quantity, result)
            sell_orderbook_append(timestamp, trader_id, order_quantity, order_price)

    elif key_figs.key_figs_test == 't_semipass' and (key_figs.buy_orderbook_test == "b_fail" and key_figs.sell_orderbook_test == "s_fail"):
        if result == 'buy_order' or result == 'buy_execute':
            #D.3.1
            offset = random.randint(2 , 10) * 0.01 
            order_price = round(key_figs.market_price - offset,2)
            order_quantity = round(random.uniform(0.40, 0.75) * max_buy_quantity)

            if order_price > 0:
                bot = bot_debiting(bot, order_price, order_quantity, result)
                buy_orderbook_append(timestamp, trader_id, order_quantity, order_price)
            else:
                order_price = 0.01
                bot = bot_debiting(bot, order_price, order_quantity, result)
                buy_orderbook_append(timestamp, trader_id, order_quantity, order_price)
            
        elif result == 'sell_order' or result == 'sell_execute':
            #D.4.2
            offset = random.randint(2 , 10) * 0.01 
            order_price = round(key_figs.market_price + offset,2)
            order_quantity = round(random.uniform(0.40, 0.75) * max_sell_quantity)

            if order_price > 0:
                bot = bot_debiting(bot, order_price, order_quantity, result)
                sell_orderbook_append(timestamp, trader_id, order_quantity, order_price)
            else:
                order_price = 0.03
                bot = bot_debiting(bot, order_price, order_quantity, result)
                sell_orderbook_append(timestamp, trader_id, order_quantity, order_price)

    '''elif result == 'buy_execute' and key_figs.sell_orderbook_test == "s_pass":
        #D.5.1
        order_price = key_figs.best_ask
        order_quantity = round(random.uniform(0.10, 0.30) * max_quantity)
        input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
        bot = bot_debiting(bot, order_price, order_quantity, result)
        bot = timer_reset(bot)
        orderbook_matching(input_order, key_figs)
    
    elif result == 'sell_execute'and key_figs.buy_orderbook_test == "b_pass":
        #D.5.2
        order_price = key_figs.best_bid
        order_quantity = round(random.uniform(0.10, 0.30) * bot["Asset"])
        input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})
        bot = bot_debiting(bot, order_price, order_quantity, result)
        bot = timer_reset(bot)
        orderbook_matching(input_order, key_figs)
'''
    return bot

# this function is to be called during the initial market startup i.e. key_figs_test = 'fail'. This does not run the matching logic, but rather fills the orderbook instead.
def bot_market_interact_initial (result, bot):
    global buy_orderbook, sell_orderbook
    key_figs = market_key_figures()
    timestamp = dt.datetime.now()
    start_price = 8
    trader_id = bot["Trader_ID"]
    price_offest = round(abs(np.random.standard_normal()) * 4) * 0.01 
    max_buy_quantity = quantity_cap_check(bot["Wealth"] / start_price)
    max_sell_quantity = quantity_cap_check(bot["Asset"])

    if result == 'buy_order' or result == "buy_execute":
        order_price = round(start_price - price_offest,2)
        order_quantity = round(random.uniform(0.25, 0.45) * max_buy_quantity)
        order_val_check = order_price * order_quantity

        if order_val_check < bot["Wealth"]:
            bot = bot_debiting(bot, order_price, order_quantity, result)

            # This is a check to see if the orderbook_matching logic can be initaited i.e. if there are orders on both sides of the market
            if key_figs.key_figs_test == "o_semipass":
                input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
                orderbook_matching(input_order)
            else:
                buy_orderbook_append(timestamp, trader_id, order_quantity, order_price)
        
    elif result == 'sell_order' or result == "sell_execute":
        order_price = round(start_price + price_offest, 2)
        order_quantity = round(random.uniform(0.15, 0.35) * max_sell_quantity)
        
        if order_quantity < bot["Asset"]:           
            bot = bot_debiting(bot, order_price, order_quantity, result)
            
            # This is a check to see if the orderbook_matching logic can be initaited i.e. if there are orders on both sides of the market
            if key_figs.key_figs_test == "o_semipass":
                input_order = pd.Series({"Timestamp" : timestamp, "Trader_ID" : trader_id, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})
                orderbook_matching(input_order)  
            else:
                sell_orderbook_append(timestamp, trader_id, order_quantity, order_price)
    return bot

# Writing results to excel file
def file_write(file_name): 
    global df_participants, transaction_log, buy_orderbook, sell_orderbook
    final_output_path = os.path.join(folder_path, file_name)
    df_participants.to_excel(final_output_path)
    with pd.ExcelWriter(final_output_path) as writer:
        df_participants.to_excel(writer, sheet_name='Participants')
        transaction_log.to_excel(writer,sheet_name='Transaction_Log')
        buy_orderbook.to_excel(writer, sheet_name='Buy_Orderbook')
        sell_orderbook.to_excel(writer, sheet_name='Sell_Orderbook')
#####################################################################
############ End of Market Functions ################################
#####################################################################
############ Beginning of Widget Setup ##############################
#####################################################################
simulation_state = "none"
simulation_thread = None
# Creating widgets for the front end
simulation_button = widgets.ToggleButtons(
    options=["Start", "Pause", "Resume", "End"],
    disabled=False,
    value=None
)
bot_num_widget = widgets.IntText(
    value=100,
    description='Number of computer participants:',
    disabled=False,
    style={'description_width': 'initial'}
)
timer_input = widgets.Checkbox(
    value=False,
    description='Select to specify simulation run time.',
    disabled=False,
    indent=False
)
sim_time_widget = widgets.IntText(
    value=180,
    description='Simulation run time (seconds):',
    disabled=True,
    style={'description_width': 'initial'}
)
file_input = widgets.Checkbox(
    value=False,
    description='Select for output of results to file.',
    disabled=False,
    indent=False
)
file_name_input = widgets.Text(
    value="",
    desciprion='Enter file name (saved to comms folder) e.g. Output.xlsx:',
    disabled=True,
    style={'description_width': 'initial'}
)
def timer_check_change(change):
    sim_time_widget.disabled = not change['new']
timer_input.observe(timer_check_change, names='value')
def file_check_change(change):
    file_name_input.disabled = not change['new']
file_input.observe(file_check_change, names='value')

grouped_widgets = [widgets.VBox([bot_num_widget, timer_input, sim_time_widget, file_input, file_name_input])]
sim_setup = widgets.Accordion(
    children=(grouped_widgets),
    titles=(['Simulation Setup'])
)

market_widget = widgets.HTML()
b_orderbook_widget = widgets.HTML()
s_orderbook_widget = widgets.HTML()
chart_widget = widgets.Image()
user_grid = widgets.GridspecLayout(3,5)
user_profile_widget = widgets.HTML()
user_positions_grid = widgets.GridspecLayout(4,2)
quantity_grid = widgets.GridspecLayout(2,1)

def start_simulation():
    global simulation_state, simulation_thread, pause_duration
    if simulation_state == "none" or simulation_state =="stopped":
        pause_duration = 0
        simulation_state = "running"
        simulation_thread = threading.Thread(target=simulation)
        simulation_thread.start()
        print("Simulation started.")
    

# function to update front-end widgets
def update_frontend():
    global transaction_log, buy_orderbook, sell_orderbook, df_participants
    key_figs = market_key_figures()
    html_string = """
    <html>
        <body>

            <h1 align="center">Market Price: {market_price}</h1>
            <h2 style="text-align:left;">
            Best Bid: {best_bid}
            <span style="float:right;">
                Best Ask: {best_ask}
            </span>
            </h2>
            <p align="center" style="color:SlateBlue">
                Open Price: {open_price}&emsp;&emsp;&ensp;Absolute Delta: {abs_price_mvmt}&emsp;&emsp;Percentage Delta: {per_price_mvmt}%
            </p>
            <p align="center" style="color:SlateBlue">
                Max Price: {price_max}&emsp;&emsp;&emsp;&emsp;Min Price: {price_min}&emsp;&emsp;&emsp;&emsp;Price Range: {price_range}
            </p>
            <h3 style="text-align:left; color:MediumSeaGreen">
            <i>Buy Orderbook</i>
            <span style="float:right; color:Tomato">
                <i>Sell Orderbook</i>
            </span>
            </h3>
            
        </body>
    </html>
    """.format(market_price=key_figs.market_price, 
    best_bid=key_figs.best_bid, 
    best_ask=key_figs.best_ask, 
    open_price=key_figs.open_price, 
    abs_price_mvmt=key_figs.abs_price_mvmt,
    per_price_mvmt=key_figs.per_price_mvmt,
    price_max=key_figs.price_max,
    price_min=key_figs.price_min,
    price_range=key_figs.price_range)

    vis_transaction_log = transaction_log
    vis_transaction_log["Timestamp"] = pd.to_datetime(vis_transaction_log["Timestamp"])
    vis_transaction_log["time_rounded"] = vis_transaction_log['Timestamp'].dt.floor('15s')
    price_graph_df = vis_transaction_log.groupby("time_rounded").agg({
        "Price" : ['first', 'max', 'min','last'],
        "Quantity" : 'sum'
    })
    price_graph_df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    columns_to_convert = ['Open', 'High', 'Low', 'Close']
    price_graph_df[columns_to_convert] = price_graph_df[columns_to_convert].round(2)

    csv_file_name = 'local_plot.csv'
    csv_file_path = os.path.join(folder_path, csv_file_name)

    price_graph_df.to_csv(csv_file_path)
    plot = pd.read_csv(csv_file_path, index_col=0,parse_dates=True)
    plot.index.name = "time_rounded"
    plot_name = "currentfigure.png" 
    plot_path = os.path.join(folder_path, plot_name)
    chart = mpf.plot(plot, type='candle', volume=True, savefig=plot_path)   
    file = open(plot_path, "rb")  
    image = file.read()
    chart_widget.value = image
    market_widget.value = html_string
    b_orderbook_widget.value = orderbook_vis(buy_orderbook, "b", html_flag=True)
    s_orderbook_widget.value = orderbook_vis(sell_orderbook, "s", html_flag=True)

# function to update user's profile widget
def update_user():
    global df_participants, user_positions_grid, user_profile_widget
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_asset = user_profile.iloc[0]['Asset']
    user_wealth = user_profile.iloc[0]['Wealth']  
    user_id = user_profile.iloc[0]['Trader_ID']  
    user_html = """
    <html>
        <body>
            <h2 align="center"> User profile </h2> 
            <i>&ensp; Last updated on: </i>
            <i>&emsp; {timestamp} </i>
            <br>
            <br>
            <b align="center"> &emsp;&emsp;Current Assets: {user_asset}</b>  
            <br>
            <b align="center"> &emsp;&emsp;Current Wealth: {user_wealth}</b> 
            <br>
            <sub align="center"> &emsp;&emsp;Starting Assets: 100000</sub>
            <br>
            <sub align="center"> &emsp;&emsp;Starting Wealth: 100000</sub> 
        </body>
    </html>
    """.format(timestamp=dt.datetime.now(),user_asset=user_asset, user_wealth=user_wealth)
    user_profile_widget.value = user_html

    # Checking for user orders
    user_borderbook_html = widgets.HTML()
    user_sorderbook_html = widgets.HTML()
    user_buy_orderbook = buy_orderbook[buy_orderbook["Trader_ID"] == user_id]
    user_sell_orderbook = sell_orderbook[sell_orderbook["Trader_ID"] == user_id]      
    user_buy_orderbook.drop(columns='Trader_ID')
    user_sell_orderbook.drop(columns='Trader_ID')

    # Creating the two tables to display user orders
    user_buy_orders_widget = widgets.HTML()
    ub_vis = PrettyTable()
    ub_vis.field_names = ['Timestamp','Price', 'Quantity']
    for index, row in user_buy_orderbook.iterrows():
        ub_vis.add_row([row['Timestamp'], row['Price'], row['Quantity']])
    user_buy_orders_widget.value  = ub_vis.get_html_string()

    user_sell_orders_widget = widgets.HTML()
    us_vis = PrettyTable()
    us_vis.field_names = ['Timestamp','Price', 'Quantity']
    for index, row in user_sell_orderbook.iterrows():
        us_vis.add_row([row['Timestamp'], row['Price'], row['Quantity']])
    user_sell_orders_widget.value  = us_vis.get_html_string()

    user_borderbook_html.value="""
    <html>
        <body>
            <h3 style="text-align:left; color:MediumSeaGreen">
            <i>User Buy Positions</i>
            </h3>
        </body>
    </html>
    """
    user_sorderbook_html.value="""
    <html>
        <body>
            <h3 style="text-align:left; color:Tomato">
            <i>User Sell Positions</i>
            </h3>
        </body>
    </html>
    """
    user_positions_grid[0,0] = user_borderbook_html
    user_positions_grid[0,1] = user_sorderbook_html
    user_positions_grid[1:,0] = user_buy_orders_widget
    user_positions_grid[1:,1] = user_sell_orders_widget

# triggered when pause putton is clicked 
def pause_simulation():
    global simulation_state, df_participants, buy_orderbook, sell_orderbook, user_positions_grid, user_profile_widget, pause_starttime

    if simulation_state =="running":
        simulation_state = "paused"
        print("Simulation paused.")
    pause_starttime = time.time()

    key_figs = market_key_figures()
    update_frontend()
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_asset = user_profile.iloc[0]['Asset']
    user_wealth = user_profile.iloc[0]['Wealth']
    user_id = user_profile.iloc[0]['Trader_ID']

    # Define widgets for user interaction
    order_side_widget = widgets.ToggleButtons(
        options=['Buy', 'Sell'],
        description='Order Side:',
        disabled=False,
        button_style = 'info'
    )
    order_type_widget = widgets.ToggleButtons(
        options=['Order', 'Execute'],
        description='Order Type:',
        disabled=False,
        button_style = 'primary'
    )
    price_widget = widgets.FloatText(
        value=key_figs.market_price, # market price
        description='Price:',
        step=0.01,
        disabled=False
    )
    quantity_widget = widgets.IntSlider(
        value=1,
        min=1,
        max=1,
        step=1,
        description='Quantity:',
        continuous_update=True
    )
    quantity_int = widgets.IntText(
        value=1,
        min=1,
        max=1,
        step=1,
        layout=widgets.Layout(width='100px', height='50px', margin='auto')
    )
    submit_button = widgets.Button(
        description='Place Order',
        disabled=False,
        button_style='success', 
        tooltip='Click to place order',
        icon='check' 
    )
    widgets.link((quantity_widget, 'value'), (quantity_int, 'value'))
    # Function to update the quantity slider's range
    def update_qty_slider(*args):
        price = price_widget.value
        o_type = order_type_widget.value
        o_side = order_side_widget.value

        if o_type == "Order" and o_side == "Buy":
            max_quantity = int(user_wealth / price) if price > 0 else 0.1
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
        elif o_type == "Execute" and o_side == "Buy":
            max_quantity = int(user_wealth / key_figs.best_ask) # best ask
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
        elif (o_type == "Order" or o_type == "Execute") and o_side == "Sell":
            max_quantity = int(user_asset)
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
            
    def update_price_app(*args):
        o_type = order_type_widget.value
        o_side = order_side_widget.value
        if o_type == "Execute" and o_side == "Buy":
            price_widget.value = key_figs.best_ask # best ask
            price_widget.disabled = True
        elif o_type == "Execute" and o_side == "Sell":
            price_widget.value = key_figs.best_bid # best bid
            price_widget.disabled = True
        else: 
            price_widget.disabled = False

    update_user()

    update_price_app()
    order_side_widget.observe(update_price_app, 'value')
    order_type_widget.observe(update_price_app, 'value')

    update_qty_slider()
    price_widget.observe(update_qty_slider, 'value')

    def user_debiting (price, quantity, order):
        global df_participants
        if order == "buy":
            order_value = price * quantity
            df_participants.loc[df_participants["Profile"] == "User", "Wealth"] -= order_value
            
        elif order == "sell":        
            df_participants.loc[df_participants["Profile"] == "User", "Asset"] -= quantity

    # Function to handle button click and place order
    def on_submit_button_clicked(b):
        order_side = order_side_widget.value
        order_type = order_type_widget.value
        price = price_widget.value
        quantity = quantity_widget.value

        # Logic to place the order into the market
        if order_side == "Buy" and order_type == "Order":
            input_order = {
                "Trader_ID" : user_id,
                "Quantity" : quantity,
                "Price" : price,
                "Flag" : "bid"
            }
            user_debiting(price, quantity, "buy")

        elif order_side == "Sell" and order_type == "Order": 
            input_order = {
                "Trader_ID" : user_id,
                "Quantity" : quantity,
                "Price" : price,
                "Flag" : "ask"
            }
            user_debiting(price, quantity, "sell")
            
        elif order_side == "Buy" and order_type == "Execute":
            input_order = {
                "Trader_ID" : user_id,
                "Quantity" : quantity,
                "Price" : key_figs.best_ask, # best ask
                "Flag" : "bid"
            }
            user_debiting(price, quantity, "buy")

        elif order_side =="Sell" and order_type == "Execute":
            input_order = {
                "Trader_ID" : user_id,
                "Quantity" : quantity,
                "Price" : key_figs.best_bid, # best bid
                "Flag" : "ask"
            }
            user_debiting(price, quantity, "sell")
    
        orderbook_matching(input_order)
        update_user()
        update_frontend()

    # Attach the callback function to the button
    submit_button.on_click(on_submit_button_clicked)

    # Ordering widgets into a grid
    quantity_grid[0,0] = quantity_widget
    quantity_grid[1,0] = quantity_int

    user_grid[:,0] = user_profile_widget
    user_grid[0,1] = order_side_widget
    user_grid[1,1] = order_type_widget
    user_grid[0,2] = price_widget
    user_grid[1,2] = quantity_grid
    user_grid[2,1:3] = submit_button
    user_grid[:,3:] = user_positions_grid

    # Allignment of widgets
    for w in [user_profile_widget, order_side_widget, order_type_widget, price_widget, quantity_widget]:
        w.layout.align_self = 'center'
    submit_button.layout.width = '100%'
    user_profile_widget.layout.border = '1px solid black'

def resume_simulation():
    global simulation_state, pause_starttime, pause_duration
    if simulation_state == "paused":
        simulation_state = "running"
        print("Simulaiton resumed.")
    resume_time = time.time()
    pause_duration = resume_time - pause_starttime

def stop_simulation():
    global simulation_state
    simulation_state = "stopped"
    if simulation_thread is not None:
        simulation_thread.join()
    print("Simulation stopped.")
    update_frontend()
    update_user()
#####################################################################
############################### End of Widget Setup #################
#####################################################################
############################### Market Simulation ###################
#####################################################################

def simulation():
    global buy_orderbook, sell_orderbook, transaction_log, df_participants, simulation_state, pause_duration
    participant_num = bot_num_widget.value
    duration = sim_time_widget.value
    file_write_bool = file_input.value
    file_name = "Output/" + file_name_input.value + ".xlsx"
    print("Simulation is beginning")

    buy_orderbook = pd.DataFrame(columns=["Timestamp", "Trader_ID", "Quantity", "Price"])
    sell_orderbook = pd.DataFrame(columns=["Timestamp", "Trader_ID", "Quantity", "Price"])
    transaction_log = pd.DataFrame(columns=["Timestamp", "Buy_Side_Trader_ID", "Sell_Side_Trader_ID", "Quantity", "Price", "Aggressor_ID"])    
    start_time = time.time()

    # Setup for market sentiment module
    IB_states = ["bull", "neutral", "bear"]
    p_states = [0.2, 0.6, 0.2]
    IB_market_state_duration = 0
    RI_market_state_duration = 0
    PI_market_state_duration = 0
    IB_market_state_start = time.time()
    RI_market_state_start = IB_market_state_start
    PI_market_state_start = IB_market_state_start
    IB_market_state = None
    RI_market_state = None
    PI_market_state = None
    grace_period = start_time + 10
    vis_start = grace_period - 2

    def IB_market_sentiment():
        IB_market_state = np.random.choice(a=IB_states, p=p_states)
        market_state_duration = random.randint(1, 90)
        market_state_start = time.time()
        return IB_market_state, market_state_start, market_state_duration

    def RI_market_sentiment(market_state):
        market_state_duration = random.randint(1, 60)
        market_state_start = time.time()
        if market_state == "bull":
            hyper_bull_states = ["bull", "h_bull"]
            h_bull_p = [0.6, 0.4]
            RI_market_state = np.random.choice(a=hyper_bull_states, p=h_bull_p)
        elif market_state == "bear":
            hyper_bear_states = ["bear", "h_bear"]
            h_bear_p = [0.6, 0.4]
            RI_market_state = np.random.choice(a=hyper_bear_states, p=h_bear_p)
        elif market_state == "neutral":
            RI_market_state = "neutral"
        return RI_market_state, market_state_start, market_state_duration

    def PI_market_sentiment():
        market_state_duration = random.randint(1, 60)
        market_state_start = time.time()
        return market_state_start, market_state_duration

    # resets the bot delay timer, using the inverse of the activity float (larger the number, more often they will interact), and a random integer to produce variation
    def timer_reset (bot):
        delay = round((1/bot["Activity"]) * abs(np.random.randint(5,50)))
        bot["Delay"] = delay + 1
        return bot

    # participant and user setup
    df_participants = participant_creation(participant_num)
    df_participants = user_creation(df_participants, participant_num)
    df_participants = df_participants.astype({"Trader_ID":int, "Asset":float, "Wealth":float, "Risk":float, "Activity":float, "Delay":int, "Profile":str})
    df_participants['PreAsset'] = df_participants["Asset"]
    df_participants['PreWealth']= df_participants["Wealth"]

    # Maing program run loop
    while simulation_state != "stopped":
        if simulation_state == "running":   
            # Timers for the visualiation, and sentiment modules. When timers run out, they execute the function i.e. updating vis or marjet sentiment 
            key_figs = market_key_figures()
            if (time.time() > grace_period and time.time() - vis_start > 3) and key_figs.key_figs_test == "pass":
                update_frontend()
                update_user()
                vis_start = time.time()             
            if time.time() - IB_market_state_start > IB_market_state_duration and time.time() > grace_period:
                IB_market_state, IB_market_state_start, IB_market_state_duration = IB_market_sentiment()
            if time.time() - RI_market_state_start > RI_market_state_duration and time.time() > grace_period:
                RI_market_state, RI_market_state_start, RI_market_state_duration = RI_market_sentiment(IB_market_state)
            if time.time() - PI_market_state_start > PI_market_state_duration and time.time() > grace_period:
                PI_market_state = IB_market_state
                PI_market_state_start, PI_market_state_duration = PI_market_sentiment()

            df_available = iteration_start()
            # running the bot decisions for the available bots
            for index, bot in df_available.iterrows():
                key_figs = market_key_figures()
                tree1_cancel_applicable = ["IB Trader", "Market Maker", "Retail Investor"]
                if bot["Profile"] in tree1_cancel_applicable:
                    bot = cancel_faraway_orders(bot)

                if key_figs.key_figs_test == 'fail' or key_figs.key_figs_test == 'o_semipass':
                    action, bot, state = bl.setup_bot_decision(bot, IB_market_state)
                elif key_figs.key_figs_test == 'pass' or key_figs.key_figs_test == "t_semipass":
                    if bot["Profile"] == "IB Trader":
                        action, bot, state = bl.IB_bot_decision(bot, IB_market_state, key_figs, transaction_log, buy_orderbook, sell_orderbook)
                    elif bot["Profile"] == "WM Trader":
                        action, bot, state = bl.WM_bot_decision(bot, IB_market_state, key_figs, transaction_log)
                    elif bot["Profile"] == "Market Maker":
                        action, bot, state = bl.MM_bot_decision(bot, key_figs, buy_orderbook, sell_orderbook)
                    elif bot["Profile"] == "Retail Investor":
                        action, bot, state = bl.RI_bot_decision(bot, RI_market_state, key_figs, transaction_log)
                    elif bot["Profile"] == "Private Investor":
                        action, bot, state = bl.PI_bot_decision(bot, PI_market_state, key_figs)
                
                if action != "no_decision":
                    if ((key_figs.key_figs_test == 'pass' or key_figs.key_figs_test == "t_semipass") and state == 'active'):
                        bot = bot_market_interact(action, bot)

                    elif (key_figs.key_figs_test == "t_semipass" and state == 'inactive'):
                        bot = bot_market_interact(action, bot)
                            
                    # runs during market setup i.e. orders being entered to market 
                    elif (key_figs.key_figs_test == 'fail' or key_figs.key_figs_test == 'o_semipass') and state == 'active':
                        bot = bot_market_interact_initial(action, bot)

                bot = timer_reset(bot)
                        
            '''try:
                print(orderbook_vis(buy_orderbook, "b", html_flag=False))      
                print(orderbook_vis(sell_orderbook, "s", html_flag=False))
            except:
                print(buy_orderbook)
                print(sell_orderbook)'''

            # merging the updated active bots with the original dataframe
            if len(df_available) > 0:
                merged_df = df_participants.merge(df_available, on="Trader_ID", how='left', suffixes   =('_old', '_new'))
                merged_df['Asset'] = merged_df['Asset_new'].fillna(merged_df['Asset_old'])
                merged_df['Wealth'] = merged_df['Wealth_new'].fillna(merged_df['Wealth_old'])
                merged_df['Delay'] = merged_df['Delay_new'].fillna(merged_df['Delay_old'])

                merged_df.drop(['Asset_old', 'Wealth_old', 'Delay_old', 'Asset_new', 'Wealth_new', 'Delay_new'], axis=1, inplace=True)
                df_participants.update(merged_df)

            # These two for loops look to remove stale orders, unless they are the users orders
            user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
            user_id = user_profile.iloc[0]['Trader_ID']  
            order_timeout = pause_duration + 45

            for index, order in buy_orderbook.iterrows():
                buy_expiry_check = dt.datetime.now() - order["Timestamp"]
                if user_id != order["Trader_ID"] and buy_expiry_check.seconds > order_timeout:
                    buy_orderbook.drop(index=index, inplace=True)
                    df_participants.loc[df_participants["Trader_ID"] == order[1],"Wealth"] += order["Price"] * order["Quantity"]

            for index, order in sell_orderbook.iterrows():
                sell_expiry_check = dt.datetime.now() - order["Timestamp"]
                if user_id != order["Trader_ID"] and sell_expiry_check.seconds > order_timeout:
                    sell_orderbook.drop(index=index, inplace=True)
                    df_participants.loc[df_participants["Trader_ID"] == order[1], "Asset"] += order["Quantity"]

            # counting down the delay counter
            df_participants["Delay"] = df_participants["Delay"].apply(lambda x: abs(x - 1))

            timer = time.time() - IB_market_state_start
            state_countdown = IB_market_state_duration - timer
            print(f"Current IB market state is {IB_market_state}, with {state_countdown} seconds remaining.")

            if time.time() - start_time > duration and timer_input.value == True:
                simulation_state == "stopped"
        
        elif simulation_state=="paused":
            time.sleep(1)

    if file_write_bool == True:
        file_write(file_name)
        print("File output has been written successfully.")   

def on_sim_button_change(change):  
    global simulation_state
    new_state = change['new']
    if new_state == "Start":
        start_simulation()
    elif new_state == "Pause":
        pause_simulation()
    elif new_state == "Resume":
        resume_simulation()
    elif new_state == "End":
        simulation_state = "end"
        stop_simulation()

#front-end display widgets    
simulation_button.observe(on_sim_button_change, names='value')   
display(sim_setup, simulation_button)
AppLayout(header=market_widget, 
          left_sidebar=b_orderbook_widget, 
          center=chart_widget, 
          right_sidebar=s_orderbook_widget, 
          footer=user_grid, 
          pane_widths=[3,6,1],
          pane_heights=['220px','500px','300px'],
          grid_gap='10px')

Accordion(children=(VBox(children=(IntText(value=100, description='Number of computer participants:', style=De…

ToggleButtons(options=('Start', 'Pause', 'Resume', 'End'), value=None)

AppLayout(children=(HTML(value='', layout=Layout(grid_area='header')), GridspecLayout(layout=Layout(grid_area=…

Simulation is beginning
Simulation started.
Current IB market state is None, with -0.03410601615905762 seconds remaining.
Current IB market state is None, with -0.5987377166748047 seconds remaining.
Current IB market state is None, with -0.6087706089019775 seconds remaining.
Current IB market state is None, with -0.6170141696929932 seconds remaining.
Current IB market state is None, with -0.6256778240203857 seconds remaining.
Current IB market state is None, with -0.6352248191833496 seconds remaining.
Current IB market state is None, with -0.6437609195709229 seconds remaining.
Current IB market state is None, with -0.6537988185882568 seconds remaining.
Current IB market state is None, with -0.689314603805542 seconds remaining.
Current IB market state is None, with -0.6993675231933594 seconds remaining.
Current IB market state is None, with -0.7351291179656982 seconds remaining.
Current IB market state is None, with -0.7569241523742676 seconds remaining.
Current IB market state is None,

Exception in thread Thread-8:
Traceback (most recent call last):
  File "c:\Users\ajlaw\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\ajlaw\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ajlaw\AppData\Local\Temp\ipykernel_38404\1882062876.py", line 1132, in simulation
  File "c:\Users\ajlaw\OneDrive\Documents\Trading\git\tradesim\botlogic.py", line 241, in IB_bot_decision
    result = bot_action + "_" + order_flag
UnboundLocalError: local variable 'force_flag' referenced before assignment


Current IB market state is bear, with 81.604318857193 seconds remaining.
